# Load necessary moduls and settings

In [ ]:
'''poker engine'''

# pylint: disable=E1101, E1601, W0612

import gc
import json
import mysql.connector
from pypokerengine.api.emulator import Emulator

import funcs_db

gc.collect()

In [ ]:
json_data = open(file='./settings.json', mode='r')
settings = json.load(json_data)
json_data.close()

# Prepare poker database for new dataset

In [ ]:
poker_db = mysql.connector.connect(user=settings['sql_user'],\
                                   host=settings['sql_host'],\
                                   database=settings['sql_database'])
sql_path = settings['sql_path']
database = settings['sql_database']

In [ ]:
tables = funcs_db.sql_select_tables(poker_db=poker_db, database=database)
for table in tables:
    funcs_db.sql_delete_allrows(poker_db=poker_db, database=database, table=table)

# Prepare poker game for new plays

## Basic settings

In [ ]:
emulator = Emulator()

In [ ]:
player_num = settings['player_num']
max_round = settings['max_round']
small_blind_amount = settings['small_blind_amount']
ante_amount = settings['ante_amount']
stack = settings['stack']

players_info = {
    'uuid-0': {'name': 'player0', 'stack': stack},
    'uuid-1': {'name': 'player1', 'stack': stack},
    'uuid-2': {'name': 'player2', 'stack': stack},
    'uuid-3': {'name': 'player3', 'stack': stack},
    'uuid-4': {'name': 'player4', 'stack': stack},
    'uuid-5': {'name': 'player5', 'stack': stack}
}

summary = list()
index = funcs_db.sql_games_max_id(poker_db=poker_db, database=database)

## Prepare database

In [ ]:
emulator.set_game_rule(player_num=player_num,\
                       max_round=max_round,\
                       small_blind_amount=small_blind_amount,\
                       ante_amount=ante_amount)
initial_state = emulator.generate_initial_game_state(players_info)
for i, player in enumerate(initial_state['table'].seats.players):
    funcs_db.sql_insert_games(poker_db=poker_db,\
                              database=database,\
                              index=index + 1,\
                              player_num=player_num,\
                              small_blind_amount=small_blind_amount,\
                              ante_amount=ante_amount,\
                              uuid=player.uuid,\
                              name=player.name,\
                              stack=player.stack,\
                              position=i,\
                              card1='',\
                              card2='',\
                              hand_db_format='',\
                              flop1='',\
                              flop2='',\
                              flop3='',\
                              turn='',\
                              river='',\
                              final_stack=0)

## Update database

In [ ]:
game_state, events = emulator.start_new_round(initial_state)
for i, player in enumerate(game_state['table'].seats.players):
    card_list = [player.hole_card[0].RANK_MAP[player.hole_card[0].rank] +\
                 player.hole_card[0].SUIT_MAP[player.hole_card[0].suit],\
                 player.hole_card[1].RANK_MAP[player.hole_card[1].rank] +\
                 player.hole_card[1].SUIT_MAP[player.hole_card[1].suit]
    ]
    card_list.sort()
    card1 = card_list[0]
    card2 = card_list[1]
    if card1[-1] == card2[-1]:
        hand_db_format = card1[:-1] + card2[:-1] + 's'
    else:
        hand_db_format = card1[:-1] + card2[:-1] + 'o'

    funcs_db.sql_update_games_cards(poker_db=poker_db,\
                                    database=database,
                                    index=index + 1,\
                                    uuid=player.uuid,\
                                    card1=card1,\
                                    card2=card2,\
                                    hand_db_format=hand_db_format)

## Prepare smallblind and bigblind phase

In [ ]:
nr = 0
step = 0
pot = 0
for i, player in enumerate(game_state['table'].seats.players):
    try:
        if player.serialize()[4][0]['action'] in ['SMALLBLIND', 'BIGBLIND']:
            if player.serialize()[4][0]['action'] == 'SMALLBLIND':
                amount = small_blind_amount
            else:
                amount = 2 * small_blind_amount

            funcs_db.sql_insert_history(poker_db=poker_db,\
                                        database=database,\
                                        phase='preflop',\
                                        nr=nr,\
                                        step=step,\
                                        uuid=player.serialize()[1],\
                                        position=i,\
                                        stack=settings['stack'],\
                                        # stack_range=,\
                                        pot=pot,\
                                        # pot_range=,\
                                        flop1='',\
                                        flop2='',\
                                        flop3='',\
                                        turn='',\
                                        river='',\
                                        action=player.serialize()[4][0]['action'],\
                                        amount=amount,\
                                        # amount_range=,\
                                        new_stack=settings['stack'] - amount,\
                                        # new_stack_range=,\
                                        new_pot=pot + amount,\
                                        # new_pot_range=)

            nr = nr + 1
            step = step + 1
            pot = pot + amount
    except:
        pass

In [ ]:
'''
import sys
sys.path.append('c:\\ProgramData\\Anaconda3\\Lib\\site-packages\\deuces\\')

import deuces
from deuces import Evaluator as e

board = [deuces.Card.new('Ah'), deuces.Card.new('Kd'), deuces.Card.new('Jc')]
hand = [deuces.Card.new('Qs'), deuces.Card.new('Th')]
print(board, hand)
e().evaluate(board, hand)
'''